TODO: Until we get real data, make synthetic data

#### Temporary: Synthetic Data

In [16]:
import numpy as np
np.random.seed(42)
data = np.concatenate((np.random.normal(100, 20, (100,5)),np.random.normal(3, 0.5, (100,5)), np.random.uniform(size=(100,2)), np.random.random_integers(0,1,(100,1))), axis=1)
data

/var/folders/37/f1ypddmd1cgbjxl4hrz8t5c40000gn/T/ipykernel_10750/2574581348.py:3: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead
  data = np.concatenate((np.random.normal(100, 20, (100,5)),np.random.normal(3, 0.5, (100,5)), np.random.uniform(size=(100,2)), np.random.random_integers(0,1,(100,1))), axis=1)


array([[1.09934283e+02, 9.72347140e+01, 1.12953771e+02, ...,
        1.67482582e-01, 1.04567840e-01, 1.00000000e+00],
       [9.53172609e+01, 1.31584256e+02, 1.15348695e+02, ...,
        6.36430250e-01, 7.06475726e-01, 1.00000000e+00],
       [9.07316461e+01, 9.06854049e+01, 1.04839245e+02, ...,
        3.15861448e-02, 9.36212246e-01, 1.00000000e+00],
       ...,
       [7.15549258e+01, 8.70685423e+01, 7.83690399e+01, ...,
        2.37544131e-01, 3.73251792e-01, 1.00000000e+00],
       [9.98405472e+01, 1.29598883e+02, 1.01547366e+02, ...,
        2.27269627e-01, 7.31959238e-02, 1.00000000e+00],
       [1.10778201e+02, 7.92550769e+01, 9.61932264e+01, ...,
        6.03448593e-01, 6.68212799e-01, 1.00000000e+00]])

### Preprocessing

### Modeling

### Evaluation